In [61]:
import numpy as np
from scipy.special import erf

In [77]:
def xyz_reader(file_name):
    file = open(file_name, 'r')

    number_of_atoms = 0
    atom_type = []
    atom_coordinates = []

    for idx, line in enumerate(file):
        if idx == 0:
            try:
                number_of_atoms = int(line.split()[0])
            except:
                print("file not in correct format.")
        
        if idx == 1:
            continue

        if idx != 0:
            split = line.split()
            atom = split[0]
            coordinates = [float(split[1]),
                           float(split[2]),
                           float(split[3])]
            atom_type.append(atom)
            atom_coordinates.append(coordinates)

    file.close()

    return number_of_atoms, atom_type, atom_coordinates

In [78]:
file_name = "../data/xyz/HeH.xyz"
number_of_atoms, atoms, atom_coordinates = xyz_reader(file_name)

In [79]:
atoms

['He', 'H']

In [80]:
atom_coordinates

[[0.0, 0.0, 0.0], [0.0, 0.0, 1.4632]]

In [81]:
number_of_atoms

2

In [82]:
STOnG = 3

zeta_dict = {'H': [1.24],
             'He': [2.0925],
             'Li': [2.69, 0.80],
             'Be': [3.68, 1.15],
             'B': [4.68, 1.50],
             'C': [5.67, 1.72]}

max_quantum_number = {'H': 1,
                      'He': 1,
                      'Li': 2,
                      'Be': 2,
                      'C': 2}

D = np.array([[0.444635, 0.535328, 0.154329],
              [0.700115, 0.399513, -0.0999672]])

alpha = np.array([[0.109818, 0.405771, 2.22766],
                  [0.0751386, 0.231031, 0.994203]])

B = 0
for atom in atoms:
    B += max_quantum_number[atom]


In [83]:
N = 2

charge_dict = {'H': 1,
               'He': 2,
               'Li': 3,
               'Be': 4,
               'B': 5,
               'C': 6,
               'N': 7,
               'O': 8,
               'F': 9,
               'Ne': 10}

In [84]:
def gauss_product(gauss_A, gauss_B):
    a, Ra = gauss_A
    b, Rb = gauss_B
    Ra, Rb = np.array(Ra), np.array(Rb)
    p = a + b
    diff = np.linalg.norm(Ra - Rb)**2
    N = (4 * a * b / (np.pi**2))**0.75
    K = N * np.exp(-a * b / p * diff)
    Rp = (a * Ra + b * Rb) / p
    
    return p, diff, K, Rp

In [85]:
def overlap(A, B):
    p, diff, K, Rp = gauss_product(A, B)
    prefactor = (np.pi / p)**1.5
    return prefactor * K

def kinetic(A, B):
    p, diff, K, Rp = gauss_product(A, B)
    prefactor = (np.pi / p)**1.5
    
    a, Ra = A
    b, Rb = B
    reduced_exponent = a * b / p
    return reduced_exponent * (3 - 2 * reduced_exponent * diff) * prefactor * K

In [86]:
def Fo(t):
    if t == 0:
        return 1
    else:
        return (0.5 * (np.pi / t)**0.5) * erf(t**0.5)

In [87]:
def potential(A, B, atom_idx):
    p, diff, K, Rp = gauss_product(A, B)
    Rc = atom_coordinates[atom_idx]
    Zc = charge_dict[atoms[atom_idx]]
    
    return (-2 * np.pi * Zc / p) * K * Fo(p * np.linalg.norm(Rp - Rc)**2)

In [88]:
def multi(A, B, C, D):
    p, diff_ab, K_ab, Rp = gauss_product(A, B)
    q, diff_cd, K_cd, Rq = gauss_product(C, D)
    multi_prefactor = 2 * np.pi**2.5 * (p * q * (p + q)**0.5)**-1
    
    return multi_prefactor * K_ab * K_cd * Fo(p * q / (p + q)*np.linalg.norm(Rp - Rq)**2)

In [89]:
S = np.zeros((B, B))
T = np.zeros((B, B))
V = np.zeros((B, B))
multi_electron_tensor = np.zeros((B, B, B, B))

for idx_a, val_a in enumerate(atoms):
    
    Za = charge_dict[val_a]
    Ra = atom_coordinates[idx_a]
    
    for m in range(max_quantum_number[val_a]):
        d_vec_m = D[m]
        zeta = zeta_dict[val_a][m]
        alpha_vec_m = alpha[m] * zeta**2
        
        for p in range(STOnG):
            for idx_b, val_b in enumerate(atoms):
                Zb = charge_dict[val_b]
                Rb = atom_coordinates[idx_b]
                for n in range(max_quantum_number[val_b]):
                    d_vec_n = D[n]
                    zeta = zeta_dict[val_b][n]
                    alpha_vec_n = alpha[n] * zeta**2
                    
                    for q in range(STOnG):
                        a = (idx_a + 1) * (m + 1) - 1
                        b = (idx_b + 1) * (n + 1) - 1
                        
                        S[a, b] += d_vec_m[p] * d_vec_n[q] * overlap((alpha_vec_m[p], Ra), (alpha_vec_n[p], Rb))
                        T[a, b] += d_vec_m[p] * d_vec_n[q] * kinetic((alpha_vec_m[p], Ra), (alpha_vec_n[p], Rb))
                        
                        for i in range(number_of_atoms):
                            V[a, b] += d_vec_m[p] * d_vec_n[q] * potential((alpha_vec_m[p], Ra), (alpha_vec_n[q], Rb), i)
                            
                        for idx_c, val_c in enumerate(atoms):
                            Zc = charge_dict[val_c]
                            Rc = atom_coordinates[idx_c]
                            
                            for k in range(max_quantum_number[val_c]):
                                d_vec_k = D[k]
                                zeta = zeta_dict[val_c][k]
                                alpha_vec_k = alpha[k] * zeta**2
                                
                                for r in range(STOnG):
                                    for idx_d, val_d in enumerate(atoms):
                                        Zd = charge_dict[val_d]
                                        Rd = atom_coordinates[idx_d]
                                        
                                        for l in range(max_quantum_number[val_d]):
                                            d_vec_l = D[l]
                                            zeta = zeta_dict[val_d][l]
                                            alpha_vec_l = alpha[l] * zeta**2
                                            
                                            for s in range(STOnG):
                                                c = (idx_c + 1) * (k + 1) - 1
                                                d = (idx_d + 1) * (l + 1) - 1
                                                multi_electron_tensor[a, b, c, d] += d_vec_m[p] * d_vec_n[q] * d_vec_k[r] * d_vec_l[s] * multi((alpha_vec_m[p], Ra), (alpha_vec_n[q], Rb), (alpha_vec_k[r], Rc), (alpha_vec_l[s], Rd))
                        

In [90]:
Hcore = T + V

In [91]:
evalS, U = np.linalg.eig(S)
diagS = np.dot(U.T, np.dot(S, U))
diagS_minushalf = np.diag(np.diagonal(diagS)**-0.5)
X = np.dot(U, np.dot(diagS_minushalf, U.T))

In [92]:
def SD_successive_density_matrix_elements(Ptilde, P):
    x = 0
    for i in range(B):
        for j in range(B):
            x += B**-2 * (Ptilde[i, j] - P[i, j])**2
    
    return x**0.5

In [93]:
P = np.zeros((B, B))
P_previous = np.zeros((B, B))
P_list = []

threshold = 100
while threshold > 10**-10:
    
    G = np.zeros((B, B))
    for i in range(B):
        for j in range(B):
            for x in range(B):
                for y in range(B):
                    G[i, j] += P[x, y] * (multi_electron_tensor[i, j, y, x] - 0.5 * multi_electron_tensor[i, x, y, j])
    Fock = Hcore + G

    Fockprime = np.dot(X.T, np.dot(Fock, X))
    evalFockprime, Cprime = np.linalg.eig(Fockprime)
    
    idx = evalFockprime.argsort()
    evalFockprime = evalFockprime[idx]
    Cprime = Cprime[:, idx]
    
    C = np.dot(X, Cprime)
    
    for i in range(B):
        for j in range(B):
            for a in range(int(N/2)):
                P[i, j] = 2 * C[i, a] * C[j, a]
                
    P_list.append(P)
    
    threshold = SD_successive_density_matrix_elements(P_previous, P)
    P_previous = P.copy()

In [94]:
print('STO3G Restricted Closed Shell HF algorithm took {} iterations to converge'.format(len(P_list)))
print('\n')
print('The orbital energies are {} and {} Hartrees'.format(evalFockprime[0], evalFockprime[1]))
print('\n')
print(f'The orbital matrix is: \n\n{C}')
print('\n')
print(f'The density/bond order matrix is: \n\n{P}')

STO3G Restricted Closed Shell HF algorithm took 6 iterations to converge


The orbital energies are -0.6072595434455743 and 1.7553131710838055 Hartrees


The orbital matrix is: 

[[ 0.40122123  0.86987445]
 [ 0.64360079 -0.70953368]]


The density/bond order matrix is: 

[[0.32195695 0.5164526 ]
 [0.5164526  0.82844396]]
